In [1]:
from pyquil import Program, get_qc
import qulacs as q
from qulacs.gate import *
from qulacs import PauliOperator
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_file
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_text
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.transforms import get_sparse_operator
from qulacs.observable import create_split_observable, create_observable_from_openfermion_text
from qulacs import Observable, QuantumStateGpu, QuantumCircuit
import pyquil.api as api
import numpy as np
import math

The process of creating a hamiltonian in qulacs is much less straightforward than with pyquil. Qulacs uses openfermion text to create the hamiltonian, which is why this cell block is much longer. It can also be read in from a text file.

In [2]:
# hopping term: a_p^dagger a_q
U1 = 1
U2 = 1
e1 = 0
e2 = 0
N = 4
n_qubits = 4
depth = 4
hamt_text = """
(-0.5 + 0j)[X0 X1] + 
(-0.5 + 0j)[X2 X3] + 
(-0.5 + 0j([Y0 Y1] + 
(-0.5 + 0j)[Y2 Y3]
"""

hamt = create_quantum_operator_from_openfermion_text(hamt_text)

# interaction term: a_p^dagger a_q^dagger a_q a_p
hamu_text = """
(0.25 + 0j) [I3] - 
(0.25 + 0j) [Z0] - 
(0.25 + 0j) [Z2] + 
(0.25 + 0j) [Z0 Z2] + 
(0.25 + 0j) [I0] - 
(0.25 + 0j) [Z1] - 
(0.25 + 0j) [Z3] +
(0.25 + 0j) [Z1 Z3]
"""
hamU = create_quantum_operator_from_openfermion_text(hamu_text)

# number term: a_p^dagger a_p
hame_text = """
(2.0 + 0j) [I] - 
(1.0 + 0j) [Z0] - 
(1.0 + 0j) [Z2] - 
(0.0 + 0j) [I] -
(0.0 + 0j) [Z1] -
(0.0 + 0j) [Z3]
"""
hame = create_quantum_operator_from_openfermion_text(hame_text)

hamiltonian_text = """
(-0.5+0j) [X0 X1] + 
(-0.5+0j) [X2 X3] + 
(-0.5+0j) [Y0 Y1] + 
(-0.5+0j) [Y2 Y3] +
(0.25+0j) [I3] - 
(0.25+0j) [Z0] - 
(0.25+0j) [Z2] + 
(0.25+0j) [Z0 Z2] + 
(0.25+0j) [I0] - 
(0.25+0j) [Z1] - 
(0.25+0j) [Z3] +
(0.25+0j) [Z1 Z3]
"""

hamiltonian_text = """
(0.5+0j)[I] + 
(-0.25+0j)[Z0] + 
(-0.25+0j)[Z2] + 
(0.25+0j)[Z0 Z2] + 
(-0.25+0j)[Z1] + 
(-0.25+0j)[Z3] + 
(0.25+0j)[Z1 Z3] + 
(-0.5+0j)[X0 X1] + 
(-0.5+0j)[Y0 Y1] + 
(-0.5+0j)[X2 X3] + 
(-0.5+0j)[Y2 Y3]
"""
hamiltonian = create_observable_from_openfermion_text(hamiltonian_text)

In [3]:
# from grove.pyvqe.vqe import VQE
# from scipy.optimize import minimize
# import numpy as np

# vqe_inst = VQE(minimizer=minimize, minimizer_kwargs={'method': 'nelder-mead'})

In [4]:
# from qucochemistry.utils import pyquilpauli_to_qubitop
# from openfermion.transforms import get_sparse_operator
# h = get_sparse_operator(pyquilpauli_to_qubitop(hamiltonian))
# np.savetxt('h.txt', np.real(h.todense()), '%-2d', ', ')
# if h.shape[0] > 1024:
#     [w, _] = sp.sparse.linalg.eigsh(h, k=1)
# else:
#     [w, _] = np.linalg.eigh(h.todense())
# w

The ansatz program is mostly the same as with pyquil when creating the circuit. 

In [5]:
 def var_ansatz(params):    
    # put initial state in the sector
    
    # Product state -- 1/sqrt(2) (|1001> + |0110>)
    c = QuantumCircuit(4)
    c.add_H_gate(0)
    c.add_CNOT_gate(0, 1)
    c.add_CNOT_gate(1, 2)
    c.add_X_gate(1)
    c.add_CNOT_gate(2, 3)
    c.add_X_gate(2)
    # |1001>
    #p.inst(X(0))
    #p.inst(X(3))
    
    # HF state -- 1/2 (|0101> + |0110> + |1001> + |1010>)

    for i in range(S):
        # building U_u
        c.add_RZ_gate(0, U1*params[3*i]/4)
        c.add_RZ_gate(1, U2*params[3*i]/4)
        c.add_RZ_gate(2, U1*params[3*i]/4)
        c.add_RZ_gate(3, U2*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_RZ_gate(2, -U1*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_CNOT_gate(1, 3)
        c.add_RZ_gate(3, -U2*params[3*i]/4)
        c.add_CNOT_gate(1, 3)
        
        #building U_\epsilon
        # c.add_RZ_gate(1, e2*params[3*i+2])
        # c.add_RZ_gate(3, e2*params[3*i+2])
        # c.add_RZ_gate(0, e1*params[3*i+2])
        # c.add_RZ_gate(2, e1*params[3*i+2])
        
        # building U_t
        c.add_H_gate(0)
        c.add_H_gate(1)
        c.add_H_gate(2)
        c.add_H_gate(3)
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_RZ_gate(1, params[3*i+1])
        c.add_RZ_gate(3, params[3*i+1])
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_H_gate(0)
        c.add_H_gate(1)
        c.add_H_gate(2)
        c.add_H_gate(3)
        c.add_RX_gate(0, -np.pi/2)
        c.add_RX_gate(1, -np.pi/2)
        c.add_RX_gate(2, -np.pi/2)
        c.add_RX_gate(3, -np.pi/2)
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_RZ_gate(1, params[3*i+1])
        c.add_RZ_gate(3, params[3*i+1])
        c.add_CNOT_gate(0, 1)
        c.add_CNOT_gate(2, 3)
        c.add_RX_gate(0, -np.pi/2)
        c.add_RX_gate(1, -np.pi/2)
        c.add_RX_gate(2, -np.pi/2)
        c.add_RX_gate(3, -np.pi/2)
        
        # building U_u
        c.add_RZ_gate(0, U1*params[3*i]/4)
        c.add_RZ_gate(1, U2*params[3*i]/4)
        c.add_RZ_gate(2, U1*params[3*i]/4)
        c.add_RZ_gate(3, U2*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_RZ_gate(2, -U1*params[3*i]/4)
        c.add_CNOT_gate(0, 2)
        c.add_CNOT_gate(1, 3)
        c.add_RZ_gate(3, -U2*params[3*i]/4)
        c.add_CNOT_gate(1, 3)
        
    return c

Here I define the VQE with qulacs version. It is a lot different than with pyquil/grove, and I am not sure how to specify the optimization method yet to nelder-mead, which may be part of why the results are different.

In [6]:
def cost(x):
    state = QuantumStateGpu(N)
    circuit = var_ansatz(x)
    circuit.update_quantum_state(state)
    return hamiltonian.get_expectation_value(state)

This code is mostly the same as the other H2VQE code, with the adjustments for qulacs' VQE.

In [7]:
from scipy.optimize import minimize
import random

def expectation(x):
    """
    Expectation value for parameters, x
    """
    return cost(x)

def scipy_minimize(x):
    """
    Minimizes expectation value using parameters found in greedy search as starting point
    Powell's conjugate direction method
    """
    return minimize(expectation, x, method='powell')

def greedy_noisy_search(initial_state):
    """
    Slightly perturb the values of the points, accepting whenever this results in a lower energy
    Total of 150 evaulations of the energy
    Change step size after 30 evaluations based on number of acceptances in previous trial group
    """
    params = initial_state
    min_energy = cost(initial_state)

    def random_point(dim, step):
        """
        Generates a random point on the perimeter of a circle with radius, step
        """
        coords = [random.gauss(0, 1) for i in range(dim)]
        norm = math.sqrt(sum([i**2 for i in coords]))
        coords = [(i / norm) * step for i in coords]
        return coords

    step = 0.1
    acceptances = 0

    # Five groups of 30 trials
    for i in range(5):
        acceptances = 0
        for j in range(30):
            
            # Slightly perturb the values of the points
            coords = random_point(len(initial_state), step)
            temp_params = [sum(x) for x in zip(params, coords)]
            
            # Calculate expectation value with new parameters
            temp_energy = cost(temp_params)
            # print(temp_params)
            
            # Greedily accept parameters that result in lower energy
            if (temp_energy < min_energy):
                min_energy = temp_energy
                params = temp_params
                acceptances += 1
        # Update step size
        step *= (acceptances / 15)

    return {'x': params, 'energy': min_energy, 'step': step}

def global_variational(S):
    """
    Complete a round of greedy and Powell's optimization for six randomly chosen points
    Further optimizes the best point
    """    
    points = [np.random.rand(S*3)*0.2 for i in range(6)]
    greedy_results = [greedy_noisy_search(x) for x in points]
    print("Done with greedy search")
    results = [scipy_minimize(i['x']) for i in greedy_results]
    print("Done with Powell's")
    res = min(results, key=lambda x:x.fun)
    # print(res)
    # For the chosen point, we continue optimizing until we cannot find improvement
    res_copy = res
    energy = res_copy.fun
    while (True):
        result = greedy_noisy_search(res_copy.x)
        temp_res = scipy_minimize(result['x'])
        temp_energy = temp_res.fun
        # print(temp_energy)

        # Optimizing will usually find an energy ~1e-6 lower
        # Stop it eventually
        tolerance = 0.001
        if (abs(temp_energy - energy) > tolerance):
            res_copy = temp_res
            energy = temp_energy
        else:
            break
    return res_copy

In [8]:
# make sure to change this S to match the one in the ansatz
S = 2
res = global_variational(S)
print(res)

Done with greedy search
Done with Powell's
   direc: array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])
     fun: array(-1.56155281)
 message: 'Optimization terminated successfully.'
    nfev: 201
     nit: 2
  status: 0
 success: True
       x: array([ 0.45238018,  0.38515119,  5.1649527 ,  1.30710574, -0.08568869,
        5.21068439])


The output here is the part that I am having trouble understanding. I am not sure what the nfev and nit variables represent. I believe the x array are the energy values, which are different than the pyquil output. Assuming all is running correctly, the execution of the global_variational function is much quicker when running on the GPU, which is a good sign. However, the output appears to be the ground state, so I think that it is not working.

I have not gotten to redefining the rest of the code yet, as I want to make sure everything up to this point is running correctly.

In [9]:
def op(s1, s2):
    # returns a PauliSum representing a_s1^daggar a_s2
    def sigma_plus(s):
        # returns sigma^+ operator on side s
        # sigma^+ = 1/2(X + iY)
        return PauliTerm('X', s, 0.5) + PauliTerm('Y', s, complex(0.0+0.5j))
    def sigma_minus(s):
        # returns sigma^- operator on side s
        # sigma^- = 1/2(X - iY)
        return PauliTerm('X', s, 0.5) - PauliTerm('Y', s, complex(0.0+0.5j))
    
    # for all sites in between site one (s1) and site two (s2), multiply by sigma z
    z = sI()
    for i in range(s1+1, s2):
        z *= PauliTerm('Z', i)
        
    return sigma_minus(s1) * z * sigma_plus(s2)